In [5]:
import os
import sys
import tensorflow as tf

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from ml.models.three_d import voxnet

In [55]:
import os
import csv
import numpy as np
import random
from keras.preprocessing.image import ImageDataGenerator
from google.cloud import storage

BLACKLIST = []


class ChunkGenerator(object):
    def __init__(self, dims=(32, 32, 32), batch_size=16,
                 shuffle=True,
                 validation=False,
                 test=False, split_test=False,
                 split=0.2):
        self.dims = dims
        self.batch_size = batch_size
        self.validation = validation

        self.datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.1,
            height_shift_range=0.1,
            zoom_range=[1.0, 1.1],
        )

        # Access Google Cloud Storage
        gcs_client = storage.Client.from_service_account_json(
            '/home/amy/credentials/client_secret.json'
        )
        bucket = gcs_client.get_bucket('elvos')

        # Get label data from Google Cloud Storage
        blob = storage.Blob('augmented_annotated_labels.csv', bucket)
        blob.download_to_filename('/home/amy/elvo-analysis/augmented_annotated_labels.csv')
        prelim_label_data = {}
        with open('/home/amy/elvo-analysis/augmented_annotated_labels.csv', 'r') as pos_file:
            reader = csv.reader(pos_file, delimiter=',')
            for row in reader:
                if row[1] != 'Unnamed: 0':
                    prelim_label_data[row[1]] = int(row[2])

        # Get 12096 random negatives from the label data to feed into our generator
        negative_counter = 0
        negative_label_data = {}
        print("getting 12096 random negative labels")
        while negative_counter < 1200:
            id_, label = random.choice(list(prelim_label_data.items()))
            if label == 0:
                negative_label_data[id_] = label
                del prelim_label_data[id_]
                negative_counter += 1
                
        # Get all of the positives from the label data
        label_data = {}
        print('getting positive labels')
        for id_, label in list(prelim_label_data.items()):
            if label == 1:
                label_data[id_] = label
        
        # Put pos and neg together into one dictionary
        label_data.update(negative_label_data)

        # Get positive chunks
        print('getting positive chunks')
        pos_blobs = bucket.list_blobs(prefix='chunk_data/normal/positive')
        files = []
        for blob in pos_blobs:
            file = blob.name

            # Check blacklist
            blacklisted = False
            for each in BLACKLIST:
                if each in file:
                    blacklisted = True
                    
            file_id = blob.name.split('/')[-1]
            file_id = file_id.split('.')[0]

            if not blacklisted:
                # Add all data augmentation methods
                files.append({
                    "name": file,
                })

        print("getting negative chunks")
        # Get negative chunks that were chosen to be in the labels
        neg_blobs = bucket.list_blobs(prefix='chunk_data/normal/negative')
        for blob in neg_blobs:
            file = blob.name

            # Check blacklist
            blacklisted = False
            for each in BLACKLIST:
                if each in file:
                    blacklisted = True

            file_id = blob.name.split('/')[-1]
            file_id = file_id.split('.')[0]

            if file_id in negative_label_data and not blacklisted:
                files.append({
                    "name": file,
                })

        # convert labels from dict to np array
        print('converting labels from dict to np array')
        labels = np.zeros(len(files))
        for i, file in enumerate(files):
            filename = file['name']
            filename = filename.split('/')[-1]
            filename = filename.split('.')[0]
            labels[i] = label_data[filename]

        # Take into account shuffling
        if shuffle:
            tmp = list(zip(files, labels))
            random.shuffle(tmp)
            files, labels = zip(*tmp)
            labels = np.array(labels)

        # Split based on validation
        print('splitting based on validation split')
        if validation:
            if split_test:
                files = files[:int(len(files) * split / 2)]
                labels = labels[:int(len(labels) * split / 2)]
            else:
                files = files[:int(len(files) * split)]
                labels = labels[:int(len(labels) * split)]
        elif test:
            if split_test:
                files = files[int(len(files) * split / 2):
                              int(len(files) * split)]
                labels = labels[int(len(labels) * split / 2):
                                int(len(labels) * split)]
            else:
                raise ValueError('must set split_test to True if test')
        else:
            files = files[int(len(files) * split):]
            labels = labels[int(len(labels) * split):]
        print(np.shape(files))
        print(np.shape(labels))
        print("Negatives: {}".format(np.count_nonzero(labels == 0)))
        print("Positives: {}".format(np.count_nonzero(labels)))
        self.files = files
        self.labels = labels
        self.bucket = bucket

    def generate(self):
        steps = self.get_steps_per_epoch()
        # print(steps)
        while True:
            for i in range(steps):
                # print(i)
                # print("D")
                x, y = self.__data_generation(i)
                yield x, y

    def get_steps_per_epoch(self):
        return len(self.files) // self.batch_size

    def __data_generation(self, i):
        bsz = self.batch_size
        files = self.files[i * bsz:(i + 1) * bsz]
        labels = self.labels[i * bsz:(i + 1) * bsz]
        images = []

        # Download files to tmp/npy/
        for i, file in enumerate(files):
            file_id = file['name'].split('/')[-1]
            file_id = file_id.split('.')[0]
            print(file_id)
            blob = self.bucket.get_blob(file['name'])
            blob.download_to_filename('/home/amy/data/chunks/{}.npy'
                                      .format(file_id))
            img = np.load('/home/amy/data/chunks/{}.npy'.format(file_id))
            print(np.shape(img))
            images.append(img)
            print("images shape: " + str(len(images)))

        images = np.ndarray(images)
        print("final images shape: " + str(np.shape(images)))
        
        # print("Loaded entire batch.")
        # print(np.shape(images))
        return images, labels



In [56]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [57]:
train_gen = ChunkGenerator()
val_gen = ChunkGenerator(validation=True)

DEBUG:urllib3.util.retry:Converted retries value: 3 -> Retry(total=3, connect=None, read=None, redirect=None, status=None)
DEBUG:google.auth.transport.requests:Making request: POST https://accounts.google.com/o/oauth2/token
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): accounts.google.com:443
DEBUG:urllib3.connectionpool:https://accounts.google.com:443 "POST /o/oauth2/token HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.googleapis.com:443
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos?projection=noAcl HTTP/1.1" 200 338
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /download/storage/v1/b/elvos/o/augmented_annotated_labels.csv?alt=media HTTP/1.1" 200 11851901
getting 12096 random negative labels
getting positive labels
('02SNCL4L74HH1N2V25', 0)
('02SNCL4L74HH1N2V26', 0)
('02SNCL4L74HH1N2V27', 0)
('02SNCL4L74HH1N2V28', 0)
('02SNCL4L74HH1N2V29', 0)
('02SNCL4L74HH1N2V32', 0)
('

DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?projection=noAcl&prefix=chunk_data%2Fnormal%2Fpositive HTTP/1.1" 200 872564
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjVjaHVua19kYXRhL25vcm1hbC9wb3NpdGl2ZS80OFhPNlVRWkk3NjZGOFRONDM2XzIzLm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fpositive HTTP/1.1" 200 872540
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjVjaHVua19kYXRhL25vcm1hbC9wb3NpdGl2ZS84S1pNTjg0QjRSNVJQUDJRMjQ1XzE2Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fpositive HTTP/1.1" 200 872440
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjRjaHVua19kYXRhL25vcm1hbC9wb3NpdGl2ZS9CV0RSQUY1Wk9RUEZPT0w3MTQ3XzkubnB5&projection=noAcl&prefix=chunk_data%2Fnormal%2Fpositive HTTP/1.1" 200 872468
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvo

DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS8yV1JIQkVETjlPV1ZWVlpYNjE3Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861158
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS8zNUtTTzIxMjI2UThXQ1FBNzkzLm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861323
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS8zQUhTMVNZQ1NOT0JJRFU1MTMwLm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861152
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS8zQkdJSzhPREk1TDZHQ1c0MTI0Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 862798


DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS83NjBOMTFRUkEzTDcxNjZaMzY5Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861522
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS83NktaTUpTMjNUT0lQVUxNNjU5Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861537
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS83QlpYMDQ2RjFVR0tDUk4wNjE4Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 862260
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS83RTZFSzdWVlNOQUVTVllTMjc0Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861389


DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9CQ1lNMlJHVVEyVzBIS0VIMzQxLm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 862603
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9CSENNRzdBM0U1WTEzVDBKMTQxLm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861460
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9CSFlaTUxMN0dCWlpMQUNXNjM0Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861457
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9CSkkyM01VQVBNVU1JRUQyMjY5Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861191


...

DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjFjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9RTFlQSFZWVTNSRzQzT1pPMTgubnB5&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 860963
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9RUEtFWk9CM1g2QjZITUNSNTc1Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861294
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjFjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9RVFQyVDBIUEY1Mk04QjdLNTIubnB5&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861322
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjFjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9SMEdVSUwxRDdROFVVTEgyMjgubnB5&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861279
DEBUG:urllib3.connection

DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9UME9OMTdYSTlEVEpEOEtPMTE5Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861498
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9UQ0Q1TTZVUUQ0RkJMNk5NMTE0Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861578
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjFjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9URkNMT0hONFJPRVlGNlFZMzcubnB5&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861310
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9URlY0NDcyR0tQWUQyNjlOMjc3Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861410
DEBUG:ur

DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9XMDk4VU41WTUwSUkwMkJYMTEyLm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861367
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9XQVAzMjdaWEY2RkdWNUU0MzM5Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861172
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9XQlpZOVMzV0RERzBZTDlZMjM0Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861214
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9XRElZSE5NQkFMWjRaUDJBMTIyLm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861526


DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9aNzlBRTZCNDdZU0dCSVRMMzY1Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 863436
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjNjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9aOTJQTEpUTTZHUDlOWDFCMjM5MC5ucHk%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861927
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9aOVNRVVNZNFhQUFNQNE5ENDUyLm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861292
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9aQU5VNEdQUFpOSUZSTjZXMjU5Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861316
DE

KeyError: '02SNCL4L74HH1N2V330'

In [ ]:
model = voxnet.VoxNet(1, "chunk weights")

In [22]:
# Train model on dataset
# model.fit_generator(generator=train_gen.generate(),
#                    validation_data=val_gen.generate(),
#                    steps_per_epoch=training_generator.get_steps_per_epoch(),
#                    validation_steps=validation_generator.get_steps_per_epoch(),
#                    epochs = 15)

model.fit(generator = train_gen.generate(), 
          samples_per_epoch = train_gen.get_steps_per_epoch(),
          nb_epoch = 15, valid_generator = val_gen.generate(), 
          nb_valid_samples = val_gen.get_steps_per_epoch(), 
          verbosity = 2)

INFO:root:Start training


/home/amy/elvo-analysis/ml/models/three_d/voxnet.py:166: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  nb_val_samples=nb_valid_samples,
/home/amy/elvo-analysis/ml/models/three_d/voxnet.py:166: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(generator=<generator..., verbose=2, callbacks=[<keras.ca..., validation_data=<generator..., steps_per_epoch=1209, epochs=15, validation_steps=302)`
  nb_val_samples=nb_valid_samples,


GET5PY3CH6RJV3UW132_18Epoch 1/15

DEBUG:urllib3.connectionpool:Resetting dropped connection: www.googleapis.com
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o/chunk_data%2Fnormal%2Fpositive%2FGET5PY3CH6RJV3UW132_18.npy HTTP/1.1" 200 837
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /download/storage/v1/b/elvos/o/chunk_data%2Fnormal%2Fpositive%2FGET5PY3CH6RJV3UW132_18.npy?generation=1531338263525281&alt=media HTTP/1.1" 200 65664
8LL3IE6PJ9GC2JX0619
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o/chunk_data%2Fnormal%2Fnegative%2F8LL3IE6PJ9GC2JX0619.npy HTTP/1.1" 200 825
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /download/storage/v1/b/elvos/o/chunk_data%2Fnormal%2Fnegative%2F8LL3IE6PJ9GC2JX0619.npy?generation=1531263057221166&alt=media HTTP/1.1" 200 65664
ZQYME7XWFWQXXOXA1351
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o/chunk_da

ValueError: could not broadcast input array from shape (32,32,32) into shape (32)